In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
from google.colab import drive

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
drive.mount('/content/drive')

Found GPU at: /device:GPU:0
Mounted at /content/drive


In [ ]:
# Read, then decode for single file.
path_to_file = '/content/drive/MyDrive/data2/data5.txt'
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

# #Read, then decode for multiple files
# story_path = r"/content/drive/MyDrive/data2/"

# def read_all_stories(story_path):
#     txt = []
#     for _, _, files in os.walk(story_path):
#         for file in files:
#             with open(os.path.join(story_path, file), encoding='utf-8') as f:
#                 for line in f:
#                     line = line.strip()
#                     if line == '----------':
#                         break
#                     if line != '':
#                         txt.append(line)
#     return txt

# text = read_all_stories(story_path)
# print("number of lines = ", len(text))


Length of text: 20702 characters


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# # Load your Gujarati text dataset (replace 'data.txt' with your dataset file)
# with open(path_to_file, 'r', encoding='utf-8') as file:
#     text = file.read()

# Split the text into sentences (assuming one sentence per line)
sentences = text.split('\n')
# sentences = text
# Tokenize and pad the sentences
max_words = 1000000  # Maximum number of words to keep in the vocabulary
max_sequence_length = 10  # Maximum sequence length for padding

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [ ]:
x = padded_sequences[:, :-1]
y = padded_sequences[:, 1:]

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length-1))
model.add(LSTM(128, return_sequences=True))
model.add(Dense(max_words, activation='softmax'))


In [ ]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# Train the model
model.fit(x, y, epochs=10, batch_size=8)
# model.save('/content/drive/MyDrive/LSTM(D3)', save_format='tf')

Epoch 1/10
29/29 [==============================] - 11s 269ms/step - loss: 13.5172 - accuracy: 0.5068
Epoch 2/10
29/29 [==============================] - 7s 242ms/step - loss: 9.9576 - accuracy: 0.5283
Epoch 3/10
29/29 [==============================] - 6s 192ms/step - loss: 5.5603 - accuracy: 0.5283
Epoch 4/10
29/29 [==============================] - 6s 213ms/step - loss: 4.4446 - accuracy: 0.5283
Epoch 5/10
29/29 [==============================] - 6s 198ms/step - loss: 4.0496 - accuracy: 0.5283
Epoch 6/10
29/29 [==============================] - 5s 187ms/step - loss: 3.8675 - accuracy: 0.5283
Epoch 7/10
29/29 [==============================] - 6s 196ms/step - loss: 3.7743 - accuracy: 0.5283
Epoch 8/10
29/29 [==============================] - 5s 183ms/step - loss: 3.7060 - accuracy: 0.5283
Epoch 9/10
29/29 [==============================] - 6s 188ms/step - loss: 3.6631 - accuracy: 0.5283
Epoch 10/10
29/29 [==============================] - 5s 178ms/step - loss: 3.6277 - accuracy: 0.52

In [ ]:
# Generate sentences
seed_text = "પેલું તોફાની ઘેટું ઘણી વાર રાજાના રસોડામાં ઘૂસી જાય"
seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]

generated_sequence = seed_sequence
print(seed_sequence)
for _ in range(5):  # Generate 20 words/symbols
    next_word_probs = model.predict(np.array([generated_sequence]))[0][-1]
    sampled_word_index = np.random.choice(len(next_word_probs), p=next_word_probs)
    generated_sequence.append(sampled_word_index)


[1039, 203, 103, 963, 134, 126, 294, 159, 90]
1/1 [==============================] - 0s 36ms/step


In [ ]:
# Convert the generated sequence back to text
generated_text = ' '.join([tokenizer.index_word.get(idx, '') for idx in generated_sequence if idx != 0])

print(generated_text)


પેલું તોફાની ઘેટું ઘણી વાર રાજાના રસોડામાં ઘૂસી જાય એક ‘હવે શિકારી
